In [67]:
import datetime as dt
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
import numpy as np
from pyproj import Proj
from google.colab import files

In [68]:
def get_goes_dataset(year, month, day, hour, product="ABI-L2-CMIPF", sat="goes-east", band="12"):
    date = dt.datetime(year, month, day)
    jday = date.timetuple().tm_yday

    syr = str(year).zfill(4)
    sjd = str(jday).zfill(3)
    shr = str(hour).zfill(2)

    fs = s3fs.S3FileSystem(anon=True)

    if sat == "goes-east":
      files = fs.glob(f"s3://noaa-goes16/{product}/{syr}/{sjd}/{shr}/*C{band}*.nc")
    else:
      files = fs.glob(f"s3://noaa-goes17/{product}/{syr}/{sjd}/{shr}/*C{band}*.nc")

    if len(files) == 0:
        raise ValueError("No GOES files found. Check date/hour/band.")

    print(f"Found {len(files)} files for {year}-{month}-{day} {hour}Z, Band {band}")

    file_objs = [fs.open(f) for f in files]
    ds = xr.open_mfdataset(file_objs, combine="nested", concat_dim="time")

    ds = ds.rename({"t": "time"}).reset_coords()

    return ds

In [69]:
def add_lat_lon(ds):
    p = ds.goes_imager_projection
    h = p.perspective_point_height + p.semi_major_axis
    lon_0 = p.longitude_of_projection_origin
    sweep = p.sweep_angle_axis
    a = p.semi_major_axis
    b = p.semi_minor_axis

    proj = Proj(proj="geos", h=h, lon_0=lon_0, sweep=sweep, a=a, b=b)

    X, Y = np.meshgrid(ds['x'].values, ds['y'].values)
    lon, lat = proj(X * h, Y * h, inverse=True)

    ds = ds.assign_coords(
        lon=(("y", "x"), lon),
        lat=(("y", "x"), lat)
    )
    return ds

In [70]:
def trim_region(data, min_lat=18, max_lat=40, min_lon=-102, max_lon=-83):
    subset = data.where(
        (data.lat > min_lat) & (data.lat < max_lat) &
        (data.lon > min_lon) & (data.lon < max_lon)
    )

    mask = ~np.isnan(subset.values)
    rows = np.where(mask.any(axis=1))[0]
    cols = np.where(mask.any(axis=0))[0]

    trimmed = subset.isel(
        y=slice(rows.min(), rows.max()),
        x=slice(cols.min(), cols.max())
    )
    return trimmed

In [71]:
def plot_hurricane(data, title="Hurricane Image"):
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(
        data.lon, data.lat, data,
        cmap="inferno", shading="auto"
    )
    plt.colorbar(label="Brightness Temperature (K)")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(title)
    plt.show()

In [72]:
def export_csv(data, filename="output.csv"):
    df = data.to_dataframe(name="CMI").reset_index()
    df = df.dropna(subset=["CMI"])
    df.to_csv(filename, index=False)
    files.download(filename)
    return df

In [ ]:
ds = get_goes_dataset(
    year=2021, month=8, day=29, hour=14,
    band="12", product="ABI-L2-CMIPF", sat="goes-east"
)

ds1 = get_goes_dataset(
    year=2021, month=8, day=29, hour=19,
    band="12", product="ABI-L2-CMIPF", sat="goes-east"
)

ds = add_lat_lon(ds)
ds1 = add_lat_lon(ds1)

ida = ds.CMI.isel(time=0)
ida1 = ds1.CMI.isel(time=0)

ida_trim = trim_region(ida)
ida_trim1 = trim_region(ida1)

plot_hurricane(ida_trim, title="Hurricane Ida — GOES-16 Band 13")
plot_hurricane(ida_trim1, title="Hurricane Ida — GOES-16 Band 13")
#export_csv(ida_trim, "ida_trim.csv")

In [ ]:
timestamps = [
    (2021, 8, 28, 6),
    (2021, 8, 28, 9),
    (2021, 8, 28, 12),
    (2021, 8, 28, 15),
    (2021, 8, 28, 18),
    (2021, 8, 28, 21),

    (2021, 8, 29, 0),
    (2021, 8, 29, 3),
    (2021, 8, 29, 6),
    (2021, 8, 29, 9),
    (2021, 8, 29, 12),
    (2021, 8, 29, 15),
    (2021, 8, 29, 18),
    (2021, 8, 29, 21),

    (2021, 8, 30, 0),
    (2021, 8, 30, 3)
]

for i, (yr, mo, dy, hr) in enumerate(timestamps):
    print(f"\n=== Processing Frame {i+1}/16 — {yr}-{mo}-{dy} {hr}Z ===")

    ds = get_goes_dataset(
        year=yr, month=mo, day=dy, hour=hr,
        band="12", product="ABI-L2-CMIPF", sat="goes-east"
    )
    ds = add_lat_lon(ds)
    data = ds.CMI.isel(time=0)
    trimmed = trim_region(data)
    filename = f"ida_{yr}{mo:02d}{dy:02d}_{hr:02d}Z.csv"
    export_csv(trimmed, filename)
    print(f"Completed frame {i+1}/16.")
print("All 16 frames processed and exported!")